Download the **files**


In [0]:
from google.colab import files
uploaded = files.upload()

Saving sales_bi_exam.csv to sales_bi_exam.csv
Saving stores_bi_exam.csv to stores_bi_exam.csv


## Sales - see the variables 


In [0]:
import pandas as pd
import io

df_sales = pd.read_csv(io.StringIO(uploaded['sales_bi_exam.csv'].decode('utf-8')))
df_sales.head()

,zip,location,month,products,total,discount
0,01000,Store A,01/01/2018,1,2500,0
1,01000,Store A,01/01/2018,1,1800,300
2,01000,Store A,01/01/2018,2,3600,150
3,01000,Store A,01/01/2018,1,1800,0
4,01000,Store A,01/03/2018,1,1800,0


In [0]:
df_sales.describe()

,products,total,discount
count,12674.000000,12674.000000,12674.000000
mean,1.117485,2164.241755,84.397191
std,0.378969,968.280955,97.649341
min,1.000000,700.000000,0.000000
25%,1.000000,1800.000000,0.000000
50%,1.000000,1800.000000,0.000000
75%,1.000000,2300.000000,150.000000
max,3.000000,10000.000000,300.000000


🧹**  the files **



In [0]:
df_stores = pd.read_csv(io.StringIO(uploaded['stores_bi_exam.csv'].decode('utf-8')))
df_stores

,location,latitud,longitud
0,Store B,19.419919,-99.168557
1,Store A,19.430864,-99.192393


Libraries 

In [0]:
df_distribution=df_sales.merge(df_stores, left_on='location', right_on='location')
df_distribution.head()

,zip,location,month,products,total,discount,latitud,longitud
0,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393
1,01000,Store A,01/01/2018,1,1800,300,19.430864,-99.192393
2,01000,Store A,01/01/2018,2,3600,150,19.430864,-99.192393
3,01000,Store A,01/01/2018,1,1800,0,19.430864,-99.192393
4,01000,Store A,01/03/2018,1,1800,0,19.430864,-99.192393


Create a net result variable based on total - discount 


In [0]:
df_distribution['net']=df_distribution['total']- df_distribution['discount']

In [0]:
df_distribution.head()

,zip,location,month,products,total,discount,latitud,longitud,net
0,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500
1,01000,Store A,01/01/2018,1,1800,300,19.430864,-99.192393,1500
2,01000,Store A,01/01/2018,2,3600,150,19.430864,-99.192393,3450
3,01000,Store A,01/01/2018,1,1800,0,19.430864,-99.192393,1800
4,01000,Store A,01/03/2018,1,1800,0,19.430864,-99.192393,1800


General descrition of the stores a/B

In [0]:
df_distribution.groupby('location')['products'].sum()

location
Store A    12170
Store B     1993
Name: products, dtype: int64

In [0]:
df_distribution.groupby('location')['discount'].sum()

location
Store A    919350
Store B    150300
Name: discount, dtype: int64

In [71]:
df_distribution.groupby('location')['total'].sum()

location
Store A    23576150
Store B     3853450
Name: total, dtype: int64

In [0]:
df_distribution.groupby('location')['net'].sum()

location
Store A    22656800
Store B     3703150
Name: net, dtype: int64

In [73]:
df_distribution.groupby('location')['zip'].nunique()

location
Store A    945
Store B    162
Name: zip, dtype: int64

In [74]:
df_distribution['zip'].nunique()

1002

In [0]:
dups_zip = df_distribution.pivot_table(index=['zip','location'], aggfunc='size')

In [79]:
dups_zip

zip    location
01000  Store A        7
01010  Store A        3
       Store B        4
01020  Store A       13
       Store B        5
                   ... 
97305  Store A        2
98615  Store A        3
99020  Store A        2
x      Store A     2754
       Store B      613
Length: 1107, dtype: int64

In [80]:
dups_zip.to_csv('df_zip.csv')
files.download('df_zip.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


User ZIP CODE ANALYSIS 


In [0]:
!pip install pgeocode

In [0]:
 import pgeocode

Testing user zip_code

In [0]:
nomi = pgeocode.Nominatim('mx')

In [0]:
result=nomi.query_postal_code('01000')

In [0]:
result.latitude
result.longitude

-99.2033

In [0]:
df_distribution['zip']

0        01000
1        01000
2        01000
3        01000
4        01000
         ...  
12669        x
12670        x
12671        x
12672        x
12673        x
Name: zip, Length: 12674, dtype: object

In [0]:
clean_zip = df_distribution.loc[df_distribution.zip != 'x', ['zip','location','month','products','total','discount','latitud','longitud','net']]
clean_zip.head()

,zip,location,month,products,total,discount,latitud,longitud,net
0,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500
1,01000,Store A,01/01/2018,1,1800,300,19.430864,-99.192393,1500
2,01000,Store A,01/01/2018,2,3600,150,19.430864,-99.192393,3450
3,01000,Store A,01/01/2018,1,1800,0,19.430864,-99.192393,1800
4,01000,Store A,01/03/2018,1,1800,0,19.430864,-99.192393,1800


In [0]:
vector=pd.DataFrame(columns=['user_lat','user_long','zip'])

In [0]:
i=0
for x in clean_zip['zip']:
   result=nomi.query_postal_code(x)
   user_lat=result.latitude 
   user_long=result.longitude
   vector['user_lat'].loc[i]=user_lat
   vector['user_long'].loc[i]=user_long
   vector['zip'].loc[i]=x
   i=i+1



In [0]:
df=pd.DataFrame()

In [0]:
df['lat_use']=vector.user_lat
df['long_use']=vector.user_long
df['zip']=vector.zip

In [0]:
user_zip=clean_zip.merge(df, left_on='zip', right_on='zip')

In [0]:
user_zip.head()

,zip,location,month,products,total,discount,latitud,longitud,net,lat_use,long_use
0,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500,19.3587,-99.2033
1,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500,19.3587,-99.2033
2,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500,19.3587,-99.2033
3,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500,19.3587,-99.2033
4,01000,Store A,01/01/2018,1,2500,0,19.430864,-99.192393,2500,19.3587,-99.2033


In [0]:
user_zip.describe()

,products,total,discount,latitud,longitud,net,lat_use,long_use
count,3.707917e+06,3.707917e+06,3.707917e+06,3.707917e+06,3.707917e+06,3.707917e+06,3.699695e+06,3.699695e+06
mean,1.108603e+00,2.153354e+03,8.409575e+01,1.943024e+01,-9.919103e+01,2.069259e+03,1.943395e+01,-9.919144e+01
std,3.672728e-01,9.336499e+02,9.768339e+01,2.544818e-03,5.542100e-03,9.380819e+02,1.146891e-01,2.036328e-01
min,1.000000e+00,7.000000e+02,0.000000e+00,1.941992e+01,-9.919239e+01,4.000000e+02,1.618620e+01,-1.169963e+02
25%,1.000000e+00,1.800000e+03,0.000000e+00,1.943086e+01,-9.919239e+01,1.650000e+03,1.943430e+01,-9.919330e+01
50%,1.000000e+00,1.800000e+03,0.000000e+00,1.943086e+01,-9.919239e+01,1.800000e+03,1.943430e+01,-9.919330e+01
75%,1.000000e+00,2.300000e+03,1.500000e+02,1.943086e+01,-9.919239e+01,2.200000e+03,1.943430e+01,-9.919330e+01
max,3.000000e+00,1.000000e+04,3.000000e+02,1.943086e+01,-9.916856e+01,9.700000e+03,3.261690e+01,-8.686099e+01


In [69]:
user_zip.to_csv('df.csv')
files.download('df.csv')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 35328, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil